# Homework 1
Download data at https://github.com/MatejFrnka/DataMining/tree/master/HW1
folder data contains 200 news articles
folder data 2 contains 930 news articles

## Runtimes:
For the smaller dataset (folder data), minhash function (see at the bottom of the document) performed the best:
* Naive: ~0.71s
* Minhash: ~0.58s
* LSH: ~0.64

For the larger dataset (folder data 2), the lower asymptotic complexity of LSH makes LSH perform the fastest:
* Naive: ~15s
* Minhash: ~4.5s
* LSH: ~3.5s



## Imports

In [14]:
import numpy as np
from time import time
from data import load_data
import itertools

MAX_INT = 2 ** 31 - 1

## Shingling
Separate string into k long shingles

In [2]:
class Shingling:

    def __init__(self, k):
        """
        :param k: Size of shingle
        """
        self.k = k

    def get_shingles(self, string):
        """
        :param string: String to turn into shingles
        :return: Set of hashed shingles
        """
        shingles = set()
        for i in range(len(string) - self.k + 1):
            # Hash string to save space and make it easier to work with
            shingles.add(hash(string[i:i + self.k]))
        return shingles


## Compute Jaccard similarity
Simply follow the formula of jaccard similarity

In [3]:
class CompareSets:
    def jaccard_similarity(self, a, b) -> float:
        """
        :param a: Set of elements
        :param b: Set of elements
        :return: Number between 0 and 1 representing the similarity in percentage. 1=100% similar sets
        """
        return len(a.intersection(b)) / len(a.union(b))

## Min hash algorithm

In [4]:

class MinHashing:

    def __init__(self, hasher_cnt):
        """
        :param hasher_cnt: Amount of hashing functions, the higher the number is, the higher the compute time and accuracy
        """
        self.hasher_cnt = hasher_cnt
        # hashing function is x*multiplier + bias % mod
        # get random multipliers for each of the hashing functions
        self.multiplier = np.random.randint(1, MAX_INT, size=hasher_cnt)
        # get random bias for each of the hashing functions
        self.bias = np.random.randint(0, MAX_INT, size=hasher_cnt)
        # get mod, higher mod = fewer collisions
        self.mod = MAX_INT

    def get_document_matrix(self, shingle_sets):
        """
        :param shingle_sets: Array of sets of shingles, every element in array represents a document
        :return: Matrix with columns representing each document and with rows for every unique element in the input shingle_sets. Matrix contains True if document contains the row's element and False if it doesn't
        """
        all_elements_array = list(set().union(*shingle_sets))
        document_matrix = []
        for i in range(len(shingle_sets)):
            # run for every document
            # create array for every element in all_elements containing True if document contains shingle and False if not
            document_matrix.append(
                np.array([k in shingle_sets[i] for k in all_elements_array]))
        return np.array(document_matrix).T

    def apply_hashers(self, val):
        """
        :param val: Value to apply hashers to
        :return: array of outputs of the hashes
        """
        return np.array(
            (np.repeat(val, self.hasher_cnt) * self.multiplier + self.bias) %
            self.mod)

    def get_signature(self, document_matrix):
        """
        :param document_matrix: Document matrix
        :return: Signature of the document matrix for each hashing function
        """
        # initiate signature with maximum values
        res_signature = np.ones((document_matrix.shape[1], self.hasher_cnt)) * MAX_INT
        for i, row in enumerate(document_matrix):
            # simulate permuting by using different hash functions
            hashed_i = self.apply_hashers(i)
            for k, col in enumerate(row):
                # if column of the document_matrix contains True, we can update the signature
                if col:
                    # update signature if new value is smaller than the existing
                    res_signature[k] = np.minimum(res_signature[k], hashed_i)
        return res_signature

    def minhash(self, shingles_sets):
        """
        :param shingles_sets: Array of sets of shingles, every element in array represents a document
        :return: Matrix document count x hashers count containing signature for every hashing function
        """
        document_matrix = self.get_document_matrix(shingles_sets)
        return self.get_signature(document_matrix)

## Compare signatures

In [5]:
class CompareSignatures:
    def compare(self, a, b):
        """
        :param a: Array of signatures of given document
        :param b: Array of signatures of given document
        :return: Percentage of matching items
        """
        return np.sum(a == b) / len(a)


## LSH

In [6]:
class LSH:

    def sep_siganture(self, sig, bands):
        """
        :param sig: Matrix of documents and their signatures
        :param bands: Amount of bands to split array into
        :return: Array of smaller matrixes of size document count x signature count / bands
        """
        return np.array_split(sig, bands, axis=1)

    def get_candidate_pairs(self, signature, band_cnt, bucket_cnt=10_000):
        """
        :param signature: Matrix of signatures (documents x signature)
        :param band_cnt: Number of bands
        :param bucket_cnt: Number of buckets, higher number decreases collisions but increases runtime
        :return: Dictionary with pairs as keys and count of their similar bands
        """
        bands = self.sep_siganture(signature, band_cnt)
        # Generate buckets to put distribute documents to
        buckets = [[[] for _ in range(bucket_cnt)] for _ in range(band_cnt)]

        for i, band in enumerate(bands):
            for k, doc in enumerate(band):
                # distribute to buckets based on band content
                buckets[i][hash(str(doc)) % bucket_cnt].append(k)

        # get candidate pairs
        candidate_pairs = dict()
        # iterate all buckets and keep those, with at least two elements
        for band in buckets:
            for slot in band:
                if len(slot) > 1:
                    # more than 1 document in a bucket, documents might be similar
                    for c in itertools.combinations(slot, 2):
                        c_frozen = frozenset(c)
                        if c_frozen in candidate_pairs:
                            candidate_pairs[c_frozen] += 1
                        else:
                            candidate_pairs[c_frozen] = 1

        return candidate_pairs

    def lsh(self, signature, band_cnt, similarity, bucket_cnt=1_000):
        """
        :param signature: Matrix of signatures (documents x signature)
        :param band_cnt: Number of bands
        :param similarity: Similarity threshold
        :param bucket_cnt: Number of buckets, higher number decreases collisions but increases runtime
        :return: Pairs of documents with similarity above the threshold
        """
        # get candidate pairs
        candidate_pairs = self.get_candidate_pairs(signature, band_cnt, bucket_cnt)
        # similarity threshold
        row_cnt = signature.shape[1] // band_cnt
        print(f"LSH tuned for {round((1 / band_cnt) ** (1 / row_cnt) * 100, 1)}% similarity")
        res = []
        comparator = CompareSignatures()
        # Compare only similar pairs
        for pair in candidate_pairs:
            p = list(pair)
            if comparator.compare(signature[p[0]], signature[p[1]]) > similarity:
                res.append(p)
        return res

## Run different ways of finding similarity
naive_similarity: uses jaccard similarity to compare sets of shingles
minhash_similarity: uses minhash to find signatures and then compares every signature of every document with every other document
lsh: finds candidate pairs and then compares their signatures

In [30]:
# helper function
def filter_results(data, similarities, threshold):
    filtered = filter(lambda x: x[0] > threshold, similarities)
    sorted_sim = sorted(filtered, key=lambda x: x[0])
    return [(data[k[1][0]][0], data[k[1][1]][0]) for k in sorted_sim]


def naive_similarity(threshold, shingle_len):
    data = load_data(-1)
    shingler = Shingling(shingle_len)
    # get shingles
    data_shingles = [shingler.get_shingles(doc[1]) for doc in data]
    similarities = []
    # compare every document with every other
    for i, doc_i in enumerate(data_shingles):
        for o, doc_o in enumerate(data_shingles):
            if i > o:
                # write down similarity
                similarities.append((CompareSets().jaccard_similarity(doc_i, doc_o), (i, o)))
    # filter above threshold
    return filter_results(data, similarities, threshold)


def minhash_similarity(threshold, shingle_len, hasher_cnt):
    data = load_data(-1)
    shingler = Shingling(shingle_len)
    similarities = []
    # get shingles
    data_shingles = [shingler.get_shingles(doc[1]) for doc in data]
    minhash = MinHashing(hasher_cnt)
    # get signatures
    signature = minhash.minhash(data_shingles)
    # compare every signature with every other signature
    for i, val_i in enumerate(signature):
        for o, val_o in enumerate(signature):
            if i > o:
                similarities.append((CompareSignatures().compare(val_i, val_o), (i, o)))
    return filter_results(data, similarities, threshold)


def lsh(threshold, shingle_len, hasher_cnt):
    data = load_data()
    shingler = Shingling(shingle_len)
    data_shingles = [shingler.get_shingles(doc[1]) for doc in data]
    minhash = MinHashing(hasher_cnt)
    signature = minhash.minhash(data_shingles)
    # run LSH
    res = LSH().lsh(signature, 10, threshold, 100)
    # transform into tuples
    return [(data[r[0]][0], data[r[1]][0]) for r in res]


In [37]:
# Define similarity parameters
sim = 0.80
shing = 3
hashers = 100

In [47]:
t = time()
print(naive_similarity(sim, shing))
print(time() - t)

[('reut2-47.txt', 'reut2-34.txt'), ('reut2-85.txt', 'reut2-82.txt'), ('reut2-146.txt', 'reut2-134.txt')]
0.7182106971740723


In [48]:
t = time()
print(minhash_similarity(sim, shing, hashers))
print(time() - t)

[('reut2-47.txt', 'reut2-34.txt'), ('reut2-85.txt', 'reut2-82.txt'), ('reut2-146.txt', 'reut2-134.txt')]
0.5832540988922119


In [50]:
t = time()
print(lsh(sim, shing, hashers))
# for small amount of documents is slower than minhash but asymptotically, it is faster
# use folder data 2 in https://github.com/MatejFrnka/DataMining/tree/master/HW1 for more documents
print(time() - t)

LSH tuned for 79.4% similarity
[('reut2-134.txt', 'reut2-146.txt'), ('reut2-85.txt', 'reut2-82.txt'), ('reut2-34.txt', 'reut2-47.txt')]
0.6449651718139648
